In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
from ipywidgets import widgets
from collections import Counter

In [2]:
# Parameters

# dataset = 'infectious'
# beta = 1
# Delta_T = 0.1 # Fraction of dataset to select (this is then divided by phi)
# k = 10
# phi = 0.1
# lines = 0
# runs = 3

def get_data(dataset, Delta_T, phi):
    lines = 0
    if dataset == "infectious":
        lines = 17298
    elif dataset == "ht09_contact":
        lines = 20818
    elif dataset == "SFHH":
        lines = 70261
    elif dataset == "tij":
        lines = 78249

    total_edges = int(lines * Delta_T)
    random_start_point = np.random.uniform(0, float(1 - Delta_T))
    start = int(lines * random_start_point)

    train_lines = int(total_edges * float(1 - phi))
    mid = start + train_lines
    end = start + total_edges

    print(f'Total lines: {total_edges}, train lines {train_lines}')
    print(f'Starting at {start} till {mid} to predict till {end}')

    path_to_file = f'../data/{dataset}.txt'

    all_data = np.loadtxt(path_to_file, delimiter='\t', dtype=int)[start: end]
    train_data = all_data[:train_lines]
    predict_data = all_data[mid:]

    return all_data, train_data, predict_data, train_lines, total_edges, start, mid, end

# Check all edges are unique

# a = np.array([src, dst, ts])
# a = a.T
# # np.any(np.all(np.unique(a, axis=1) == a, axis=1))
# u, c = np.unique(a, return_counts=True, axis=0)
# u[c > 1]

In [3]:
# Construct Graph
def make_graph(all_data):
    src, dst, ts = [], [], []
    graph = nx.MultiGraph()

    for line in all_data:
        src.append(line[0])
        dst.append(line[1])
        ts.append(line[2])

        if graph.nodes.get(line[0]) is None:
            graph.add_node(line[0], is_infected=False, infected_at=math.inf, influence=-1, influences=None, pred=None, polyfit_prediction=None)
        if graph.nodes.get(line[1]) is None:
            graph.add_node(line[1], is_infected=False, infected_at=math.inf, influence=-1, influences=None, pred=None, polyfit_prediction=None)

        graph.add_edge(line[0], line[1], key=line[2], timestamp=line[2])

    src, dst, ts = np.array(src), np.array(dst), np.array(ts) # ts is given sorted
    print(graph.nodes.get(src[0]))

    return src, dst, ts, graph

In [4]:
def check_edge(edge, g, inf, ins, beta):
    # TODO: use beta
    if g.nodes.get(edge[0])['is_infected'] and edge[2] > g.nodes.get(edge[0])['infected_at'] and not g.nodes.get(edge[1])['is_infected']:
        inf += 1
        g.nodes.get(edge[1])['in_infected'] = True
        g.nodes.get(edge[1])['infected_at'] = edge[2]
        ins[str(edge[2])] = ins.get(str(edge[2]), 0) + 1
    
    return inf, ins, g

def find_influence_of_node(edges, node, at, train_lines, beta):
    g = nx.Graph()
    influence, influences, pred = 0, {}, {}
    
    for edge in edges:
        if g.nodes.get(edge[0]) is None:
            g.add_node(edge[0], is_infected=False, infected_at=math.inf)
        if g.nodes.get(edge[1]) is None:
            g.add_node(edge[1], is_infected=False, infected_at=math.inf)

    g.nodes.get(node)['is_infected'] = True
    g.nodes.get(node)['infected_at'] = at

    # TODO: Check influence. I think it's getting unset somewhere because of how Python handles pointers.
    for i, edge in enumerate(edges):
        if i <= train_lines:
            influence, influences, g = check_edge(edge, g, influence, influences, beta)
            new_edge = (edge[1], edge[0], edge[2])
            influence, influences, g = check_edge(new_edge, g, influence, influences, beta)
        else:
            influence, pred, g = check_edge(edge, g, influence, pred, beta)
            new_edge = (edge[1], edge[0], edge[2])
            influence, pred, g = check_edge(new_edge, g, influence, pred, beta)
                
    return influence, influences, pred

# TODO: Fix this
def find_influence_in_graph(graph, all_data, train_lines, beta):
    for i, edge in enumerate(all_data):
        for e in [edge[0], edge[1]]:
            if graph.nodes.get(e)['influence'] == -1:
                z, zz, zzz = find_influence_of_node(all_data, e, edge[2], train_lines, beta)
                # print(f'z: {z}, zz: {zz}, zzz: {zzz}')
                graph.nodes.get(e)['influence'] = z
                graph.nodes.get(e)['influences'] = zz
                if zzz is not {}:
                    graph.nodes.get(e)['pred'] = zzz # time series of influence. [(0, 0), (1, 5), (2, 10), (4, 12)]

    most_influential_nodes = sorted(list(graph.nodes.data("influence")), key=lambda x: x[1], reverse=True)[:5]
    print(most_influential_nodes)
    return most_influential_nodes

In [86]:
def polyfit(data, new_points, dimension):
#     print(f'data: {data}')
    k, v = list(map(lambda x: int(x), data.keys())), list(data.values())
#     print(f'keys: {k}')
    print(f'values: {v}')
    
    fit = np.polyfit(k, v, 1) #The use of 1 signifies a linear fit.

    line = np.poly1d(fit)
    new_points = np.arange(new_points) + (k[-1] + 1)
    new_p = line(new_points)
    print(f'New points predicted are: {new_p}')

    return new_p
#     return np.cumsum(line(new_points))

In [87]:
# Find k most influential nodes
def simulate(dataset, delta_t, phi, beta):
    print(f'Running simulation for dataset {dataset} with delta_t {delta_t} and phi {phi} and beta {beta}')
    all_data, train_data, predict_data, train_lines, total_edges, start, mid, end = get_data(dataset, delta_t, phi)
    src, dst, ts, graph = make_graph(all_data)
    most_influential_nodes = find_influence_in_graph(graph, all_data, train_lines, beta) # 5 most influential nodes [(node id, influence)]

    for node in most_influential_nodes:
        thisnode = graph.nodes.get(node[0])
        if thisnode is not {}:
            thisnode['polyfit_prediction'] = polyfit(thisnode['influences'], new_points=5, dimension=1)
        
simulate("infectious", 0.1, 0.1, 1)

Running simulation for dataset infectious with delta_t 0.1 and phi 0.1 and beta 1
Total lines: 1729, train lines 1556
Starting at 12948 till 14504 to predict till 14677
{'is_infected': False, 'infected_at': inf, 'influence': -1, 'influences': None, 'pred': None, 'polyfit_prediction': None}
[(297, 114), (266, 100), (296, 82), (230, 69), (292, 66)]
values: [4, 2, 1, 1, 3, 2, 3, 2, 4, 3, 4, 3, 2, 4, 4, 3, 2, 2, 2, 3, 1, 3, 2, 1, 2, 3, 4, 2, 2, 1, 2, 2, 2, 1, 1, 3, 2, 2, 2, 1, 2, 2, 2, 2, 3, 2, 3, 2, 2, 1]
New points predicted are: [1.80355115 1.78756294 1.77157472 1.7555865  1.73959829]
values: [2, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 1, 2, 3, 2, 2, 1, 3, 3, 3, 2, 2, 1, 1, 2, 1, 2, 1, 1, 3, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1]
New points predicted are: [1.2740922  1.27075226 1.26741232 1.26407238 1.26073243]
values: [1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 3, 4, 3, 3, 4, 2, 2, 3, 3, 4, 3, 3, 2, 1, 2, 1, 3, 3, 2, 2, 3, 2, 2, 3, 4, 1, 1]
Ne

In [11]:
dataset_dropdown = widgets.Dropdown(options=['infectious', 'ht09_contact', 'SFHH', 'tij_InVS15'], description='Dataset', disabled=False, continuous_update=True)
phi_slider = widgets.FloatSlider(min=0.1, max=0.9, step=0.01, value=0.1, continuous_update=True)
delta_t_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.1, continuous_update=True)
beta_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.75, continuous_update=True)

w = widgets.interactive(simulate, dataset=dataset_dropdown.value, delta_t=delta_t_slider.value, phi=phi_slider.value, beta=beta_slider.value)
display(w)



interactive(children=(Text(value='infectious', description='dataset'), FloatSlider(value=0.1, description='del…

In [ ]:
# We can plot real influence with predicted influence
# Minimize on MSE or MAE or any other distance metric to find best hyper-parameters

In [7]:
#a_slider = widgets.IntSliderWidget(min=-5, max=5, step=1, value=0)
# b_slider = widgets.FloatSliderWidget(min=-5, max=5, step=0.3, value=0)
#w = widgets.interactive(sigmoid_demo,a=a_slider, b=b_slider)
#display(w)

In [ ]:
# bor = 'A'
# drop_down = widgets.Dropdown(options=['A','B','C','D'],
#                              description='Choose',
#                              disabled=False)
#
# def dropdown_handler(change):
#     global bor
#     print(change.new)
#     bor = change.new  # This line isn't working
# drop_down.observe(dropdown_handler, names='value')
# display(drop_down)